# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f520db3ae80>
├── 'a' --> tensor([[-0.0538, -2.1706,  1.0831],
│                   [ 1.5994,  0.8798,  0.6716]])
└── 'x' --> <FastTreeValue 0x7f520db3adc0>
    └── 'c' --> tensor([[ 0.6019, -0.1378, -0.7804,  0.4486],
                        [-0.4320,  2.6346,  0.6479,  1.8865],
                        [-0.5043,  1.4418,  0.1368, -1.2055]])

In [4]:
t.a

tensor([[-0.0538, -2.1706,  1.0831],
        [ 1.5994,  0.8798,  0.6716]])

In [5]:
%timeit t.a

74 ns ± 0.0756 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f520db3ae80>
├── 'a' --> tensor([[ 0.1319, -0.2635,  0.5488],
│                   [-1.3693,  1.3424, -1.1650]])
└── 'x' --> <FastTreeValue 0x7f520db3adc0>
    └── 'c' --> tensor([[ 0.6019, -0.1378, -0.7804,  0.4486],
                        [-0.4320,  2.6346,  0.6479,  1.8865],
                        [-0.5043,  1.4418,  0.1368, -1.2055]])

In [7]:
%timeit t.a = new_value

72 ns ± 0.0473 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0538, -2.1706,  1.0831],
               [ 1.5994,  0.8798,  0.6716]]),
    x: Batch(
           c: tensor([[ 0.6019, -0.1378, -0.7804,  0.4486],
                      [-0.4320,  2.6346,  0.6479,  1.8865],
                      [-0.5043,  1.4418,  0.1368, -1.2055]]),
       ),
)

In [10]:
b.a

tensor([[-0.0538, -2.1706,  1.0831],
        [ 1.5994,  0.8798,  0.6716]])

In [11]:
%timeit b.a

68.1 ns ± 0.0271 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3955,  0.8209, -0.6054],
               [ 2.3603,  0.2168,  1.0860]]),
    x: Batch(
           c: tensor([[ 0.6019, -0.1378, -0.7804,  0.4486],
                      [-0.4320,  2.6346,  0.6479,  1.8865],
                      [-0.5043,  1.4418,  0.1368, -1.2055]]),
       ),
)

In [13]:
%timeit b.a = new_value

543 ns ± 0.326 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

884 ns ± 6.32 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 23.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 735 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

147 µs ± 587 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f520dad9460>
├── 'a' --> tensor([[[-0.0538, -2.1706,  1.0831],
│                    [ 1.5994,  0.8798,  0.6716]],
│           
│                   [[-0.0538, -2.1706,  1.0831],
│                    [ 1.5994,  0.8798,  0.6716]],
│           
│                   [[-0.0538, -2.1706,  1.0831],
│                    [ 1.5994,  0.8798,  0.6716]],
│           
│                   [[-0.0538, -2.1706,  1.0831],
│                    [ 1.5994,  0.8798,  0.6716]],
│           
│                   [[-0.0538, -2.1706,  1.0831],
│                    [ 1.5994,  0.8798,  0.6716]],
│           
│                   [[-0.0538, -2.1706,  1.0831],
│                    [ 1.5994,  0.8798,  0.6716]],
│           
│                   [[-0.0538, -2.1706,  1.0831],
│                    [ 1.5994,  0.8798,  0.6716]],
│           
│                   [[-0.0538, -2.1706,  1.0831],
│                    [ 1.5994,  0.8798,  0.6716]]])
└── 'x' --> <FastTreeValue 0x7f520dad91c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.8 µs ± 73.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f513dd44b50>
├── 'a' --> tensor([[-0.0538, -2.1706,  1.0831],
│                   [ 1.5994,  0.8798,  0.6716],
│                   [-0.0538, -2.1706,  1.0831],
│                   [ 1.5994,  0.8798,  0.6716],
│                   [-0.0538, -2.1706,  1.0831],
│                   [ 1.5994,  0.8798,  0.6716],
│                   [-0.0538, -2.1706,  1.0831],
│                   [ 1.5994,  0.8798,  0.6716],
│                   [-0.0538, -2.1706,  1.0831],
│                   [ 1.5994,  0.8798,  0.6716],
│                   [-0.0538, -2.1706,  1.0831],
│                   [ 1.5994,  0.8798,  0.6716],
│                   [-0.0538, -2.1706,  1.0831],
│                   [ 1.5994,  0.8798,  0.6716],
│                   [-0.0538, -2.1706,  1.0831],
│                   [ 1.5994,  0.8798,  0.6716]])
└── 'x' --> <FastTreeValue 0x7f513dd44d00>
    └── 'c' --> tensor([[ 0.6019, -0.1378, -0.7804,  0.4486],
                        [-0.4320,  2.6346,  0.6479,  1.8865],
                 

In [23]:
%timeit t_cat(trees)

32 µs ± 94.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.1 µs ± 53.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.0538, -2.1706,  1.0831],
                [ 1.5994,  0.8798,  0.6716]],
       
               [[-0.0538, -2.1706,  1.0831],
                [ 1.5994,  0.8798,  0.6716]],
       
               [[-0.0538, -2.1706,  1.0831],
                [ 1.5994,  0.8798,  0.6716]],
       
               [[-0.0538, -2.1706,  1.0831],
                [ 1.5994,  0.8798,  0.6716]],
       
               [[-0.0538, -2.1706,  1.0831],
                [ 1.5994,  0.8798,  0.6716]],
       
               [[-0.0538, -2.1706,  1.0831],
                [ 1.5994,  0.8798,  0.6716]],
       
               [[-0.0538, -2.1706,  1.0831],
                [ 1.5994,  0.8798,  0.6716]],
       
               [[-0.0538, -2.1706,  1.0831],
                [ 1.5994,  0.8798,  0.6716]]]),
    x: Batch(
           c: tensor([[[ 0.6019, -0.1378, -0.7804,  0.4486],
                       [-0.4320,  2.6346,  0.6479,  1.8865],
                       [-0.5043,  1.4418,  0.1368, -1.2055]],
         

In [26]:
%timeit Batch.stack(batches)

84 µs ± 231 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.0538, -2.1706,  1.0831],
               [ 1.5994,  0.8798,  0.6716],
               [-0.0538, -2.1706,  1.0831],
               [ 1.5994,  0.8798,  0.6716],
               [-0.0538, -2.1706,  1.0831],
               [ 1.5994,  0.8798,  0.6716],
               [-0.0538, -2.1706,  1.0831],
               [ 1.5994,  0.8798,  0.6716],
               [-0.0538, -2.1706,  1.0831],
               [ 1.5994,  0.8798,  0.6716],
               [-0.0538, -2.1706,  1.0831],
               [ 1.5994,  0.8798,  0.6716],
               [-0.0538, -2.1706,  1.0831],
               [ 1.5994,  0.8798,  0.6716],
               [-0.0538, -2.1706,  1.0831],
               [ 1.5994,  0.8798,  0.6716]]),
    x: Batch(
           c: tensor([[ 0.6019, -0.1378, -0.7804,  0.4486],
                      [-0.4320,  2.6346,  0.6479,  1.8865],
                      [-0.5043,  1.4418,  0.1368, -1.2055],
                      [ 0.6019, -0.1378, -0.7804,  0.4486],
                      [-0.4320,  

In [28]:
%timeit Batch.cat(batches)

154 µs ± 388 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

360 µs ± 4.53 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
